## Predicting All Stars Based of a Full Year of Baseball
Team Members: Chris Spartz, Kevin Gnaster, Zach Josten <br />
Team Number: 31

In [2]:
import numpy as np
import numpy.matlib
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
import scipy.stats as st
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.formula.api as smf
import statsmodels.api as sm
import math

In [3]:
star = pd.read_excel("FinalDatasetMLBallStar 11.25.21.xlsx")
star.drop(columns=['xwOBA'], inplace=True)
star=star.rename(columns={'All Star Starter?':'ASstarter'})
star.head(5)

Season              YearName              Name Team    G   PA  HR    R  \
0    2019        2019Mike Trout        Mike Trout  LAA  134  600  45  110   
1    2019      2019Alex Bregman      Alex Bregman  HOU  156  690  41  122   
2    2019  2019Christian Yelich  Christian Yelich  MIL  130  580  44  100   
3    2019    2019Cody Bellinger    Cody Bellinger  LAD  156  660  47  121   
4    2019     2019Marcus Semien     Marcus Semien  OAK  162  747  33  123   

   RBI  SB  ...    OBP    SLG   wOBA   wRC+  BsR   Off   Def  WAR  playerid  \
0  104  11  ...  0.438  0.645  0.436  177.0  7.1  65.9  -0.3  8.4     10155   
1  112   5  ...  0.423  0.592  0.418  167.0 -2.1  57.3   4.8  8.4     17678   
2   97  30  ...  0.429  0.671  0.442  174.0  8.5  65.2  -3.9  7.8     11477   
3  115  15  ...  0.406  0.629  0.415  161.0  1.4  54.2   3.4  7.7     15998   
4   92  10  ...  0.369  0.522  0.373  138.0  1.7  37.7  14.1  7.6     12533   

   ASstarter  
0          1  
1          1  
2          1  
3          1  
4          0  

[5 rows x 25 columns]

In [4]:
star2 = star.dropna(how='any',axis=0).copy()

In [5]:
star2.shape

(20161, 25)

In [6]:
star2.drop(columns=['playerid'], inplace=True)
starCorr=star2.corr()
starCorr.iloc[:,-1:].sort_values(by='ASstarter', ascending=False)

ASstarter
ASstarter   1.000000
WAR         0.413208
Off         0.402159
HR          0.352875
RBI         0.328526
R           0.319117
PA          0.264179
G           0.226927
SB          0.166729
ISO         0.153212
SLG         0.146887
wRC+        0.129601
wOBA        0.127734
OBP         0.112059
AVG         0.105985
BB%         0.073941
BABIP       0.063696
BsR         0.048647
Season      0.003286
Def        -0.016528
K%         -0.069844

In [7]:
star3 = star2.loc[star2['PA'] >= 450].reset_index()
star3.shape

(3803, 25)

In [9]:
traditional=["HR","R","RBI","SB","BB%","K%","AVG","OBP","SLG"]
advanced=["ISO","BABIP","wOBA","wRC+","BsR","Off","Def","WAR"]
allStats=["HR","R","RBI","SB","BB%","K%","AVG","OBP","SLG","ISO","BABIP","wOBA","wRC+","BsR","Off","Def","WAR"]
y=star3.ASstarter
star3[traditional]
#Training testing split is 75-25

HR    R  RBI  SB    BB%     K%    AVG    OBP    SLG
0     45  110  104  11  0.183  0.200  0.291  0.438  0.645
1     41  122  112   5  0.172  0.120  0.296  0.423  0.592
2     44  100   97  30  0.138  0.203  0.329  0.429  0.671
3     47  121  115  15  0.144  0.164  0.305  0.406  0.629
4     33  123   92  10  0.116  0.137  0.285  0.369  0.522
...   ..  ...  ...  ..    ...    ...    ...    ...    ...
3798   4   79   34  44  0.063  0.154  0.232  0.280  0.301
3799  34  104  133   6  0.082  0.127  0.298  0.354  0.541
3800  12   47   48   2  0.121  0.182  0.218  0.327  0.360
3801   2   60   35  12  0.035  0.064  0.250  0.280  0.297
3802   1   47   26   9  0.048  0.197  0.226  0.267  0.276

[3803 rows x 9 columns]